In [ ]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)

In [ ]:
import pandas as pd
import stock
from strategies import ta

def update_scan_results(scan_results_df, allowed_etfs, ib):
    for s in scan_results_df['symbol']:
        sx = stock.StockXDaily(ib, s)
        sx.req_fundamentals(max_days_old=1)  # gets the fundamentals from the IB API
        sx.req_ohlcv()
        fund_results = sx.get_funadmentals_validation_results(allowed_etfs)
        ta_results   = sx.get_TA_validation_results()

        # Find the index of the row with the symbol
        index = scan_results_df[scan_results_df['symbol'] == s].index[0]

        # Update the row with the validation results
        for key, value in fund_results.items():
            scan_results_df.loc[index, key] = value
        
        # Update the row with the validation results
        for key, value in ta_results.items():
            scan_results_df.loc[index, key] = value

    return scan_results_df

# Example usage
scan_results_df = pd.DataFrame({
    'rank': [1, 2, 3],
    'symbol': ['TSLA', 'AAPL', 'MSFT'],
})



updated_scan_results_df = update_scan_results(scan_results_df, ['XLK', 'XLY', 'XLI'], ib)
updated_scan_results_df

Using cached data for TSLA (age: 2024-12-22 09:34:58)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Using cached data for AAPL (age: 2024-12-22 09:44:08)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\AAPL_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\AAPL_1_day.csv
Using cached data for MSFT (age: 2024-12-22 09:44:10)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MSFT_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MSFT_1_day.csv


,rank,symbol,Sector1 Valid,Market Cap > 300M,Vol 10DayMA > 300K,Fundamentals Passed,Close > $1,Above 200MA,Above 150MA,Breaks Above 50MA,50MA Slope > 0,Gap Up > 4%,Volume > 50K,Volume Above 10MA,Volume Dev > 80%,TA Passed
0,1,TSLA,True,True,True,1.0,True,True,True,False,True,False,True,False,False,0.56
1,2,AAPL,True,True,True,1.0,True,True,True,False,True,False,True,False,False,0.56
2,3,MSFT,True,True,True,1.0,True,True,True,False,True,False,True,False,False,0.56
